### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Разработка рекомендательной модели на основе данных о товарах</p>

<a id="1.2"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Содержание</div>

<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li>
    <li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li>
    <li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span></li>
    <li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Предобработка данных</a></span></li>
    <li><span><a href="#Рекомендательная-система" data-toc-modified-id="Рекомендательная-система-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Рекомендательная система</a></span></li>
    <li><span><a href="#Тесты" data-toc-modified-id="Тесты-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Тесты</a></span></li>
    <li><span><a href="#Вывод" data-toc-modified-id="Вывод-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Вывод</a></span></li>

<a id="Описание-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Описание данных</div>

## marketing_dealer - список дилеров

## marketing_dealerprice - результат работы парсера площадок дилеров
<ul>
<li>product_key - уникальный номер позиции;</li>
<li>price - цена;</li>
<li>product_url - адрес страницы, откуда собраны данные;</li>
<li>product_name - заголовок продаваемого товара;</li>   
<li>date - дата получения информации;</li>
<li>dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)</li>
</ul>

## marketing_product - список товаров, которые производит и распространяет заказчик
<ul>
<li>article - артикул товара;</li>
<li>ean_13 - код товара (см. EAN 13);</li>
<li>name - название товара;</li>
<li>cost - стоимость;</li>
<li>min_recommended_price - рекомендованная минимальная цена;</li>
<li>recommended_price - рекомендованная цена;</li>
<li>category_id - категория товара;</li>
<li>ozon_name - названиет товара на Озоне;</li>
<li>name_1c - название товара в 1C;</li>
<li>wb_name - название товара на Wildberries;</li>
<li>ozon_article - описание для Озон;</li>
<li>wb_article - артикул для Wildberries;</li>
<li>ym_article - артикул для Яндекс.Маркета;</li>
</ul>

## marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров
<ul>
<li>key - внешний ключ к marketing_dealerprice</li>
<li>product_id - внешний ключ к marketing_product</li>
<li>dealer_id - внешний ключ к marketing_dealer</li>
</ul>

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">


Необходимо разработать рекомендательную модель, которая позволяет по продукту дилера найти подходящие соответствия из рядов товаров производителя.

</div>

<a id="Загрузка-библиотек"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Загрузка библиотек</div>

In [1]:
import re
import pandas as pd
from fuzzywuzzy import fuzz, process

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Изучим наши датафреймы.

<a id="EDA"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> EDA</div>

In [2]:
marketing_product = pd.read_csv('marketing_product.csv', sep = ';')
marketing_product.head()

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


In [3]:
marketing_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         496 non-null    int64  
 1   id                 496 non-null    int64  
 2   article            496 non-null    object 
 3   ean_13             464 non-null    float64
 4   name               494 non-null    object 
 5   cost               491 non-null    float64
 6   recommended_price  491 non-null    float64
 7   category_id        447 non-null    float64
 8   ozon_name          458 non-null    object 
 9   name_1c            485 non-null    object 
 10  wb_name            455 non-null    object 
 11  ozon_article       365 non-null    float64
 12  wb_article         340 non-null    float64
 13  ym_article         337 non-null    object 
 14  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(2), object(7)
memory usage: 58.3+ KB


In [4]:
marketing_dealerprice = pd.read_csv('marketing_dealerprice.csv', sep = ';')
marketing_dealerprice.head()

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2


In [5]:
marketing_productdealerkey = pd.read_csv('marketing_productdealerkey.csv', sep = ';')
marketing_productdealerkey.head()

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


In [6]:
manufacturer_data = marketing_product[['id','name']]

In [7]:
manufacturer_data.dtypes

id       int64
name    object
dtype: object

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Сохраним таблицу. Она необходимо, чтобы в дальнейшем не повторять этот шаг при каждом повторном вызове скрипта модели.

In [8]:
manufacturer_data.to_csv('manufacturer_data.csv', index=False)

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Загрузим объединенную таблицу.

In [9]:
manufacturer_data = pd.read_csv('manufacturer_data.csv') 

In [10]:
manufacturer_data

,id,name
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,443,"Герметик акриловый цвет сосна, ф/п 600мл"
3,147,Кондиционер для белья с ароматом королевского...
4,502,"Герметик акриловой цвет Белый, 7 кг"
...,...,...
491,127,Средство для уборки помещений после пожара с д...
492,160,"Жидкое моющее средство для стирки шерсти, шелк..."
493,74,Средство для чистки гриля и духовых шкафовCook...
494,34,Средство для мытья полов с полимерным покрытие...


In [11]:
manufacturer_data.isnull().sum()

id      0
name    2
dtype: int64

In [12]:
manufacturer_data = manufacturer_data.dropna()

<a id="Предобработка-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Предобработка данных</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Создадим ручной словарь, который поможет нам разделитель слитные слова в столбце 'name'. 

In [13]:
list_product_word = [
    'PROSEPT', 'концентрат', 'Crystal', 'готовый', 'Duty', 'Multipower', 'MULTIPOWER', 'White', 'Belizna',
    'Cooky', 'Diona', 'готовое', 'ULTRA', 'Antifoam', 'Bath', 'Universal', 
    'Carpet', 'концентрированное', 'Flox', 'эффектом', 'splash', 'epoxy', 
    'Candy', 'Optic', 'Clean', 'шампунь', 'штуки', 'Super', 'Plastix', 'Proplast', 'Ириса', 'FLOX'
    ]


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Создадим функцию, разделяющую слитные слова в столбце 'name'. 

In [14]:
def get_not_continuous_words(data): 
    '''
    
    The function separates the combined
    words in a column 'name'.
    
    '''
    result = data['name']
    for word in list_product_word:
        tmp_str = result.split(str(word))
        if len(tmp_str) > 1:
            result = tmp_str[0] + ' ' + word + ' ' + tmp_str[1]

    return result

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Создадим новый столбец 'name_split' без слитных слов. 

In [15]:
manufacturer_data['name_split'] = manufacturer_data.apply(get_not_continuous_words, axis=1)  

/var/folders/cg/b2wsqr_s7fd0x08bc2ykkndc0000gn/T/ipykernel_5291/1177491123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manufacturer_data['name_split'] = manufacturer_data.apply(get_not_continuous_words, axis=1)


In [16]:
manufacturer_data.head()

,id,name,name_split
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,Антисептик невымываемый PROSEPT ULTRA конце...
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,Антигололед - 32 PROSEPT готовый состав / 1...
2,443,"Герметик акриловый цвет сосна, ф/п 600мл","Герметик акриловый цвет сосна, ф/п 600мл"
3,147,Кондиционер для белья с ароматом королевского...,Кондиционер для белья с ароматом королевского...
4,502,"Герметик акриловой цвет Белый, 7 кг","Герметик акриловой цвет Белый, 7 кг"


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Создадим ручной словарь, который поможет нам разделитель слитные слова при вводе товара дилера. 

In [17]:
list_product_dealer = [
    'антижук', 'PROSEPT', 'universal', 'ULTRA', 'grill', 'удаления',
    'floor', 'remover', 'средство', 'стекол', 'зеркал', 'пластика',
    'акриловых', 'bath', 'acryl', 'profi', 'Eco', 'multipower', 'xm11',
    'graffiti', 'плесени', 'грибка', 'gel', 'снятия', 'shine', 'грунтовка',
    '20л', '10л', '2л', 'hand', 'cristal', 'против', 'лак', 'полуматовый',
    'глянцевый', 'невымываемый', 'машины', 'splash', 'орех', 'сlean', 'acid',
    'polish', 'удаления', 'hard', 'посуды', 'полов', 'комнат',  'spray',
    'посудомоечной', 'lime', 'rinser', 'sport', 'спортивной', 'черных',
    'black', 'сауны', 'бани', 'труб', 'засоров', 'extra', 'после',
    'очистки', 'ухода', 'мебелью', 'зеленый', 'красный', 'fungi'
    ]


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Создадим функция, которая разделяет слитные слова при вводе товара дилера. 

In [18]:
def get_not_continuous_words_when_entering(row):
    '''
    
    The function separates concatenated 
    words when entering a dealer product.
    
    '''
    result = row
    for word in list_product_dealer:
        result_split = result.split(word)
        if len(result_split) > 1:
            result = result_split[0] + ' ' + word + ' ' + result_split[1]

    return result


<a id="Рекомендательная-система"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Рекомендательная система</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Создадим рекомендательную систему.

In [19]:
def get_suitable_products(
        dealer_product: str, 
        manufacturer_products: pd.Series,
        levenshtein_distance_max: int,
    ) -> list:
    '''
    A Model Explanation System
    
    return: Array of suitable manufactur products
    '''
    
    suitable_products = []
    for product in manufacturer_products['name_split']:
        l_d = fuzz.token_sort_ratio(get_not_continuous_words_when_entering(dealer_product), product)
        if (l_d >= levenshtein_distance_max):
            manufacturer_products_id = manufacturer_products[
                manufacturer_products['name_split'] == product
            ]['id'].to_string(index=False)
            suitable_products.append({
                'id':  manufacturer_products_id, 
                'product_name': product, 
                'levenshtein_distance': l_d 
            })
    return suitable_products

In [20]:
def get_solution(
        dealer_product: str,
        length: int = 10,
        manufacturer_products: pd.Series = manufacturer_data,
        levenshtein_distance_max: int = 50,
    ) -> list:
    
    suitable_solution = get_suitable_products(
        get_not_continuous_words_when_entering(dealer_product), 
        manufacturer_products, 
        levenshtein_distance_max
    )
    solution = sorted(suitable_solution, key=lambda x: x['levenshtein_distance'], reverse=True)
    last_index = length if length < len(solution) else len(solution)
    
    return solution[0: last_index]
    

<a id="Тесты"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Тесты</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Проверим как отрабатывает модель при различных входных данных.

## Тест 1

### Случайное название товара дилера

In [21]:
get_solution('Средство для удаления ленты  клейкой ')

[{'id': '1',
  'product_name': 'Средство для удаления наклеек  Duty   Universal  / 210мл',
  'levenshtein_distance': 61},
 {'id': '129',
  'product_name': 'Средство для удаления клейкой ленты, клея, наклеек Duty   Universal  готовый  состав / 2 л',
  'levenshtein_distance': 60},
 {'id': '128',
  'product_name': 'Средство для удаления клейкой ленты, клея, наклеек Duty   Universal  готовый  состав / 400мл',
  'levenshtein_distance': 59},
 {'id': '122',
  'product_name': 'Средство для удаления гипсовой пыли Duty   White  концентрат  1:10-1:100 / 5 л  ',
  'levenshtein_distance': 52},
 {'id': '121',
  'product_name': 'Средство для удаления гипсовой пыли Duty   White  концентрат  1:10-1:100 / 1 л  ',
  'levenshtein_distance': 52},
 {'id': '408',
  'product_name': 'Средство для снятия обоев спрей / 1 л',
  'levenshtein_distance': 51}]

## Тест 2

### Случайное название товара дилера

In [22]:
get_solution('Антисептик для влажной древесины PROSEPT BiO, концентрат 1:19 20 л')

[{'id': '267',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19 / 20 л',
  'levenshtein_distance': 100},
 {'id': '268',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19  / 30 л',
  'levenshtein_distance': 98},
 {'id': '266',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19 / 5 л',
  'levenshtein_distance': 98},
 {'id': '265',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19  / 1 л',
  'levenshtein_distance': 96},
 {'id': '257',
  'product_name': 'Антисептик для внутренних работ PROSEPT  INTERIOR концентрат  1:10 / 1 л',
  'levenshtein_distance': 71},
 {'id': '243',
  'product_name': 'Отбеливатель для древесины PROSEPT  50 концентрат  1:1 / 20 л',
  'levenshtein_distance': 69},
 {'id': '258',
  'product_name': 'Антисептик для внутренних работ PROSEPT  INTERIOR концентрат  1:10 / 5 л',
  'levenshtein_distance': 69},
 {'id': '263',
  'product_name': 'Антисепт

In [23]:
len('Антисептик для влажной древесины PROSEPT BiO, концентрат 1:19 20 л')

66

In [24]:
len('Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19 / 20 л')

69

In [25]:
(66 + 69)/(66 + 69 - 5) 

1.0384615384615385

## Тест 3

### Случайное название товара дилера

In [26]:
get_solution('Удалитель мха готовый состав 1 л PROSEPT')

[{'id': '209',
  'product_name': 'Удалитель мха  готовый  состав / 1 л ',
  'levenshtein_distance': 89},
 {'id': '210',
  'product_name': 'Удалитель мха  готовый  состав / 5 л ',
  'levenshtein_distance': 86},
 {'id': '484',
  'product_name': 'Набор Удалитель мха,  готовый  состав, 1 л, 2 шт',
  'levenshtein_distance': 80},
 {'id': '403',
  'product_name': 'Удалитель ржавчины RUST REMOVER  готовый  состав / 0,5 л',
  'levenshtein_distance': 74},
 {'id': '324',
  'product_name': 'Лак для камня  PROSEPT  готовый  состав / 2 л',
  'levenshtein_distance': 70},
 {'id': '323',
  'product_name': 'Лак для камня  PROSEPT  готовый  состав / 0,9 л',
  'levenshtein_distance': 68},
 {'id': '219',
  'product_name': 'Лак яхтный  PROSEPT , полуматовый готовый  состав / 2 л',
  'levenshtein_distance': 67},
 {'id': '218',
  'product_name': 'Лак яхтный  PROSEPT , глянцевый готовый  состав / 2 л ',
  'levenshtein_distance': 67},
 {'id': '390',
  'product_name': 'Удалитель плесени FUNGI CLEAN  готовый  сос

In [27]:
len('Удалитель мха готовый состав 1 л PROSEPT')

40

In [28]:
len('Удалитель мха  готовый  состав / 1 л ')

37

In [29]:
(40 + 37) / (40 +37-11)

1.1666666666666667

## Тест 4

### C заданной длинной рекомендованных товаров 5 (по умолчанию 10)

In [30]:
get_solution('Антисептик для влажной древесины PROSEPT BiO, концентрат 1:19 20 л', length=5)

[{'id': '267',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19 / 20 л',
  'levenshtein_distance': 100},
 {'id': '268',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19  / 30 л',
  'levenshtein_distance': 98},
 {'id': '266',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19 / 5 л',
  'levenshtein_distance': 98},
 {'id': '265',
  'product_name': 'Антисептик для влажной древесины PROSEPT  BiO концентрат  1:19  / 1 л',
  'levenshtein_distance': 96},
 {'id': '257',
  'product_name': 'Антисептик для внутренних работ PROSEPT  INTERIOR концентрат  1:10 / 1 л',
  'levenshtein_distance': 71}]

In [31]:
fuzz.ratio('Привет!' , 'рривет!')

86

<a id="Вывод"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Вывод</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<br>📑Было необходимо разработать<b> рекомендательную модель, которая позволяет по продукту дилера найти подходящие соответствия из рядов товаров производителя.</b>. 
        
        
<br><b> Входные данные - данные о товарах заказчика и дилера. </b>
    
<br> 📊 Для разработки модели используется библиотека FuzzyWuzzy и расстояние Левенштейна. 
        
<br> 📊 Функция `get_not_continuous_words()` разделяет слитные слова в столбце 'name'.
        
<br> 📊 Функция `get_not_continuous_words_when_entering()` при вводе разделяет слитные слова в столбце 'name'.. 
        
<br> 📊 Функция `get_solution()`принимает строковую переменную товара дилера и возвращает список рекомендованных товаров продавца.
    
<br> ***Фичи:***
    <ul>
    <li>✅ работает для случайного набора слов в названии товара дилера;</li>
    <li>✅ отрабатывает с сырыми данными парсинга;</li> 
    <li>✅ позволяет работать с заданным значением длины возвращаемого массива товаров.</li> 
    </ul>
    
<br> 📊 Данная версия тетрадки лучше сопоставляет продукт дилера с рядом товаров производителя, так как добавлена предобработка данных. Например, при вводе "Пропитка-антисептик PROSEPT Ultra невымываемая для ответственных конструкций конц-т 1:10 1л", "Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.", "Средство универсальное Prosept Universal Spray, 500мл" старая версия тетрадки не выдавала правильного сопоставления товара, либо расстояние Левенштейна было ниже, в отличии от этой версии, так как товары производителя включали слитные слова. 

<br>Исходя из анализа, сделан вывод:
        <br>             <b>- подготовлена рекомендательная модель для сопоставления товара дилера с товарами производителя. </b>      

In [33]:
get_solution('Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.')

[{'id': '236',
  'product_name': 'Шпатлевка-замазка для заделки глубоких выбоин и трещин Plastix  / 1,4 кг',
  'levenshtein_distance': 53},
 {'id': '237',
  'product_name': 'Шпатлевка-замазка для заделки глубоких выбоин и трещин Plastix  / 5 кг',
  'levenshtein_distance': 52}]